In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [38]:
from hdfs import InsecureClient # hadoop 서버와 연결하는  client객체 모듈(hdfs 접근) !pip install hdfs
import requests # web 사용
import json # json 기능 사용
import datetime as dt
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, lit, concat_ws, when, row_number, concat
from pyspark.sql.window import Window

In [3]:
# client = InsecureClient('http://localhost:9870', user='root')

#### DataLake에 Json파일 불러오기

In [54]:
# food_info = spark.read.csv('/data/food_info.csv',header=True)

json_file_path = '/data/food_info.json'
json_data = spark.read.json(json_file_path)
food_info_data=spark.createDataFrame(json_data.select('data').first()['data'])
json_data.show()

+-----------------------------+--------------------+
|                         data|                meta|
+-----------------------------+--------------------+
|[{간장게장, 게장(-醬)은 한...|{{food, ingredien...|
+-----------------------------+--------------------+



In [55]:
from pyspark.sql.functions import explode, col
df_exploded = json_data.select("data", "meta.desc", "meta.cols").withColumn("data", explode("data"))
df_result = df_exploded.select(
    col("data.food"),
    col("data.info"),
    col("data.ingredient")
)
df_result.show()

food_info_data = food_info_data.withColumn(
    'ingredient',
    when(col('food') == '양념치킨', concat(col('ingredient'), lit(',닭고기'))).otherwise(col('ingredient'))
)

+--------+----------------------------------+--------------------------------+
|    food|                              info|                      ingredient|
+--------+----------------------------------+--------------------------------+
|간장게장|   게장(-醬)은 한국의 게 요리이...|  꽃게 5마리,간장 3머그컵,물 ...|
|  갈비찜| 갈비찜은 갈비에 양념과 간을 하...|  돼지갈비 2kg,감자,양파,대파...|
|  갈비탕|         갈비탕(영어: galbi-tan...|   갈비탕용 소고기 1kg,무 1토...|
|  감자전| 감자전은 감자를 갈아서 기름에 ...|감자 3개,물 감자가잠길만큼,소...|
|  감자탕|           감자탕(Gamja-tang, p...|   감자탕용 돼지등뼈 1.5kg,감...|
|곱창구이| 곱창은 소나 돼지의 작은창자로 ...|  곱창 1kg,감자 2개,참기름 1스푼|
|    김밥| 김밥은 소금과 참기름으로 맛을 ...|    밥,김밥용 김 12장,햄 8개,...|
|  김치전| 김치전 또는 김치부침개는 잘게 ...| 김치 2컵,부침가루 1컵,김치국...|
|김치찌개|        김치찌개(영어: Kimchi-j...|  김치 1/2포기,포도씨유 1큰술...|
|  닭갈비| 닭갈비는 한국의 대표적인 볶음 ...|    닭안심 500g,양파 1개,당근...|
|닭볶음탕|닭도리탕은 형태상 조림에 가까운...|  닭 1마리,당면,감자3개,당근1...|
|도토리묵|도토리묵은 도토리의 녹말을 이용...|  도토리가루 1컵,물 6컵,소금 ...|
|된장찌개| 된장찌개는 끓는 육수에 된장을 ...|   느타리버섯 1줌,양파 1/2개,...|
|  떡볶이|  떡볶이는 떡면(가는 가래떡)을 ...|   떡볶이떡 2컵,물 2컵,대파 1

In [56]:
food_info_data.select('ingredient').collect()[28]

Row(ingredient='물엿 4큰술,다진 마늘 2큰술,케찹 2큰술,설탕 2큰술,고추장 1큰술,고춧가루 1큰술,물 1큰술,간장 1큰술,닭고기')

In [58]:
JDBC = {
      'url':'jdbc:mysql://localhost:3306/finalPrj_DW?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigMysql',
      'password':'bigMysql1234@'   
      }
}
JDBC

{'url': 'jdbc:mysql://localhost:3306/finalPrj_DW?characterEncoding=utf8&serverTimezone=Asia/Seoul',
 'props': {'user': 'bigMysql', 'password': 'bigMysql1234@'}}

In [59]:
food_info_data.write.mode('overwrite').jdbc(url=JDBC['url'], table='food_info',properties=JDBC['props'])